# 机器学习的目标

第 4 章的 3 个例子,尽管互相从模型到输入都有一定差别,但是无一例外,没当经过几个轮次后,模型就开始过拟合.在测试集上效果开始变差.

模型的泛化与优化是机器学习的根本矛盾,而过拟合是如何产生的?如何得到泛化性能更好的模型?


## 理解过拟合

在所有能见到的模型和数据集上,都会存在过拟合问题,随着训练进行,模型在训练集上表现越来越好,但是超过一定程度模型在测试集上表现反而开始变差.如下图.

![typical_overfitting](./typical_overfitting.png)